In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset
import torchvision
import torch.nn.functional as F
from sklearn.preprocessing import scale,MinMaxScaler,Normalizer,StandardScaler
import torch.optim as optim
from Regression.CnnModel import ConvNet, DeepSpectra, AlexNet
import os
from datetime import datetime
from Evaluate.RgsEvaluate import ModelRgsevaluate, ModelRgsevaluatePro
import matplotlib.pyplot  as plt
from ray import tune
from DataLoad.DataLoad import LoadNirtest
LR = 0.001
BATCH_SIZE = 16
TBATCH_SIZE = 240


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#自定义加载数据集
class MyDataset(Dataset):
    def __init__(self,specs,labels):
        self.specs = specs
        self.labels = labels

    def __getitem__(self, index):
        spec,target = self.specs[index],self.labels[index]
        return spec,target

    def __len__(self):
        return len(self.specs)



###定义是否需要标准化
def ZspPocessnew(X_train, X_test, y_train, y_test, need=True): #True:需要标准化，Flase：不需要标准化

    global standscale
    global yscaler

    if (need == True):
        standscale = StandardScaler()
        X_train_Nom = standscale.fit_transform(X_train)
        X_test_Nom = standscale.transform(X_test)

        #yscaler = StandardScaler()
        yscaler = MinMaxScaler()
        y_train = yscaler.fit_transform(y_train.reshape(-1, 1))
        y_test = yscaler.transform(y_test.reshape(-1, 1))

        X_train_Nom = X_train_Nom[:, np.newaxis, :]
        X_test_Nom = X_test_Nom[:, np.newaxis, :]

        ##使用loader加载测试数据
        data_train = MyDataset(X_train_Nom, y_train)
        data_test = MyDataset(X_test_Nom, y_test)
        return data_train, data_test
    elif((need == False)):
        yscaler = StandardScaler()
        # yscaler = MinMaxScaler()

        X_train_new = X_train[:, np.newaxis, :]  #
        X_test_new = X_test[:, np.newaxis, :]

        y_train = yscaler.fit_transform(y_train)
        y_test = yscaler.transform(y_test)

        data_train = MyDataset(X_train_new, y_train)
        ##使用loader加载测试数据
        data_test = MyDataset(X_test_new, y_test)

        return data_train, data_test




def CNNTrain(config):




    EPOCH=20


    model1 = AlexNet(1, config['linear1'], config['linear2'], 3840)
    device = "gpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model1)
    model.to(device)


    

    criterion = nn.MSELoss().to(device)  # 损失函数为焦损函数，多用于类别不平衡的多分类问题
    optimizer = optim.Adam(model.parameters(), lr=LR)#,  weight_decay=0.001)  # 优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）
    # # initialize the early_stopping object
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, verbose=1, eps=1e-06,
                                                           patience=20)
    print("Start Training!")  # 定义遍历数据集的次数
    # to track the training loss as the model trains
    for epoch in range(EPOCH):
        train_losses = []
#         model.train()  # 不训练
        train_rmse = []
        train_r2 = []
        train_mae = []
        for i, data in enumerate(train_loader):  # gives batch data, normalize x when iterate train_loader
            inputs, labels = LoadNirtest('Rgs')     # 输入和标签都等于data
            inputs = Variable(inputs).type(torch.FloatTensor).to(device)  # batch x
            #print(inputs.shape)
            labels = Variable(labels).type(torch.FloatTensor).to(device)  # batch y
            #print(labels.shape)
            output = model(inputs)  # cnn output
            loss = criterion(output, labels)  # MSE
            optimizer.zero_grad()  # clear gradients for this training step
            loss.backward()  # backpropagation, compute gradients
            optimizer.step()  # apply gradients
            pred = output.detach().cpu().numpy()
            y_true = labels.detach().cpu().numpy()
            train_losses.append(loss.item())
            rmse, R2, mae = ModelRgsevaluatePro(pred, y_true, yscaler)
            # plotpred(pred, y_true, yscaler))
            train_rmse.append(rmse)
            train_r2.append(R2)
            train_mae.append(mae)
        avg_train_loss = np.mean(train_losses)
        avgrmse = np.mean(train_rmse)
        avgr2 = np.mean(train_r2)
        avgmae = np.mean(train_mae)
        tune.report(my_loss=avg_train_loss)

if __name__ == '__main__':
 
    config = {
        "linear1": tune.choice([3500,3000,2500]),  # 自定义采样
        "linear2": tune.choice([1000,1500,2000]),  # 从给定值中随机选择
    }
    result = tune.run(  # 执行训练过程，执行到这里就会根据config自动调参了
        CNNTrain,  # 要训练的模型
        resources_per_trial={"cpu": 2, "gpu": 1},# 指定训练资源
        config=config,
        num_samples=20,  # 迭代的次数
    )
    # 得到最后的结果
    print("======================== Result =========================")
    print(result.results_df)

(CNNTrain pid=5928) 2023-01-31 15:11:14,787	ERROR function_trainable.py:298 -- Runner Thread raised error.
(CNNTrain pid=5928) Traceback (most recent call last):
(CNNTrain pid=5928)   File "/root/miniconda3/lib/python3.8/site-packages/ray/tune/trainable/function_trainable.py", line 289, in run
(CNNTrain pid=5928)     self._entrypoint()
(CNNTrain pid=5928)   File "/root/miniconda3/lib/python3.8/site-packages/ray/tune/trainable/function_trainable.py", line 362, in entrypoint
(CNNTrain pid=5928)     return self._trainable_func(
(CNNTrain pid=5928)   File "/root/miniconda3/lib/python3.8/site-packages/ray/util/tracing/tracing_helper.py", line 466, in _resume_span
(CNNTrain pid=5928)     return method(self, *_args, **_kwargs)
(CNNTrain pid=5928)   File "/root/miniconda3/lib/python3.8/site-packages/ray/tune/trainable/function_trainable.py", line 684, in _trainable_func
(CNNTrain pid=5928)     output = fn()
(CNNTrain pid=5928)   File "/tmp/ipykernel_466/1487508134.py", line 95, in CNNTrain
(CN

Result for CNNTrain_71142_00000:
  date: 2023-01-31_15-11-14
  experiment_id: e2442cd69f134a899a58df8960bdec64
  hostname: autodl-container-095311853c-3000e9c7
  node_ip: 172.17.0.11
  pid: 5928
  timestamp: 1675149074
  trial_id: '71142_00000'
  


In [1]:
import torch
import torch.nn as nn
import numpy as np
from ray import tune


class LinearRegressionModel(nn.Module):
    def __init__(self, input_shape, linear1, linear2, output_shape):
        super(LinearRegressionModel, self).__init__()
        self.linear1 = nn.Linear(input_shape, linear1)
        self.linear2 = nn.Linear(linear1, linear2)
        self.linear3 = nn.Linear(linear2, output_shape)

    def forward(self, x):
        l1 = self.linear1(x)
        l2 = self.linear2(l1)
        l3 = self.linear3(l2)
        return l3


def train_model(config):  # 修改1：修改参数，所有的参数都要借助config传递
    # 指定参数与损失函数
    model = LinearRegressionModel(x_train.shape[1], config['linear1'], config['linear2'], 1)
    epochs = 1000  # 迭代1000次
    learning_rate = 0.01  # 学习率
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  # 优化函数
    criterion = nn.MSELoss()  # Loss使用MSE值，目标是使MSE最小

    loss_list = []
    for epoch in range(epochs):
        epoch += 1
        optimizer.zero_grad()  # 梯度清零
        outputs = model(x_train)  # 前向传播
        loss = criterion(outputs, y_train)  # 计算损失
        loss.backward()  # 返向传播
        loss_list.append(loss.detach().numpy())
        optimizer.step()  # 更新权重参数
    mean_loss = np.mean(loss_list)
    tune.report(my_loss=mean_loss)


if __name__ == '__main__':
    x_train = torch.randn(100, 4)  # 生成100个4维的随机数，作为训练集的 X
    y_train = torch.randn(100, 1)  # 作为训练集的label
    # train_model(x_train, y_train, 32, 8) # 修改2：就不需要这样启动了，注释掉这一行

    # 修改3：下面就是封装的方法
    config = {
        "linear1": tune.sample_from(lambda _: np.random.randint(2, 5)),  # 自定义采样
        "linear2": tune.choice([2, 4, 8, 16]),  # 从给定值中随机选择
    }
    result = tune.run(  # 执行训练过程，执行到这里就会根据config自动调参了
        train_model,  # 要训练的模型
        resources_per_trial={"cpu": 8, },  # 指定训练资源
        config=config,
        num_samples=20,  # 迭代的次数
    )
    # 得到最后的结果
    print("======================== Result =========================")
    print(result.results_df)



2023-01-31 15:06:27,392	WARNING utils.py:594 -- Detecting docker specified CPUs. In previous versions of Ray, CPU detection in containers was incorrect. Please ensure that Ray has enough CPUs allocated. As a temporary workaround to revert to the prior behavior, set `RAY_USE_MULTIPROCESSING_CPU_COUNT=1` as an env var before starting Ray. Set the env var: `RAY_DISABLE_DOCKER_CPU_WARNING=1` to mute this warning.
2023-01-31 15:06:27,468	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
2023-01-31 15:06:28,228	WARNING function_trainable.py:619 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Result for train_model_c8362_00000:
  date: 2023-01-31_15-06-32
  done: false
  experiment_id: 7c94a3a22418429b8386ce355f9e386d
  hostname: autodl-container-095311853c-3000e9c7
  iterations_since_restore: 1
  my_loss: 0.9916567802429199
  node_ip: 172.17.0.11
  pid: 3856
  time_since_restore: 0.4164090156555176
  time_this_iter_s: 0.4164090156555176
  time_total_s: 0.4164090156555176
  timestamp: 1675148792
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: c8362_00000
  warmup_time: 0.004136085510253906
  
Result for train_model_c8362_00000:
  date: 2023-01-31_15-06-32
  done: true
  experiment_id: 7c94a3a22418429b8386ce355f9e386d
  experiment_tag: 0_linear1=2,linear2=2
  hostname: autodl-container-095311853c-3000e9c7
  iterations_since_restore: 1
  my_loss: 0.9916567802429199
  node_ip: 172.17.0.11
  pid: 3856
  time_since_restore: 0.4164090156555176
  time_this_iter_s: 0.4164090156555176
  time_total_s: 0.4164090156555176
  timestamp: 1675148792
  timesteps_since_resto

2023-01-31 15:06:39,601	INFO tune.py:758 -- Total run time: 11.37 seconds (11.07 seconds for the tuning loop).


======================== Result =========================
              my_loss  time_this_iter_s  done timesteps_total episodes_total  \
trial_id                                                                       
c8362_00000  0.991657          0.416409  True            None           None   
c8362_00001  0.983202          0.341048  True            None           None   
c8362_00002  0.976943          0.339103  True            None           None   
c8362_00003  0.979999          0.344286  True            None           None   
c8362_00004  0.976378          0.330642  True            None           None   
c8362_00005  0.986233          0.328428  True            None           None   
c8362_00006  0.978975          0.335543  True            None           None   
c8362_00007  0.979499          0.338412  True            None           None   
c8362_00008  0.975357          0.322140  True            None           None   
c8362_00009  0.978736          0.324543  True            None 